In [28]:
import pandas as pd
import numpy as np
import xlearn as xl
import tensorflow as tf
import gc

In [77]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [67]:
test_path = "./train_test_data_models/location/video/Punjabi/is_vp_succ2/test.txt"
xlearn_test_path = "./train_test_data_models/location/video/Punjabi/is_vp_succ2/xlearn_test.txt"
model_path = "./train_test_data_models/location/video/Punjabi/is_vp_succ2/out/model.out"
predicted_file_path = "./train_test_data_models/location/video/Punjabi/is_vp_succ2/xlearn_predicted.txt"
load_model_path = "./train_test_data_models/location/video/Punjabi/is_vp_succ2/out/model.txt"
mapping_file_path = "./train_test_data_models/location/video/Punjabi/is_vp_succ2/user_post_ffm_mapping.csv"

line_counts = 200

In [68]:
# count=0
# lines = []
# with open(test_path) as f:
#     for line in f:
#         lines.append(line)
#         if(count > line_count):
#             break
#         count+=1
# with open(xlearn_test_path, "w") as f:
#     f.writelines(lines)

In [72]:
with open(xlearn_test_path) as f:
    lines = f.read().splitlines()
    
print(lines[:5])

['1 0:3422065:1 1:95477:1 2:3633926:1 ', '0 0:2429568:1 1:85714:1 2:3634564:1 ', '0 0:1888436:1 1:63919:1 2:3633849:1 ', '0 0:1760473:1 1:44696:1 2:3633926:1 ', '0 0:1675911:1 1:17563:1 2:3634063:1 ']


In [85]:
ffm_model = xl.create_ffm() 
ffm_model.setTest(xlearn_test_path)
ffm_model.setSigmoid()
ffm_model.predict(model_path, predicted_file_path)
with open(predicted_file_path) as f:
    xlearn_scores = [float(i) for i in f.readlines()]
print(xlearn_scores[:5])

[0.268623, 0.0609916, 0.0826926, 0.547922, 0.0386218]


In [73]:
model = open(load_model_path, 'r').read().splitlines()

embs = {}
for s in model:
    x = s.split(': ')
    embs[x[0]] =  list(map(float, x[1].split(' ')))
del model

In [100]:
global_bias = embs['bias'][0]
scores = []
for l in lines:
    k = l.split(':')
    feat_0, feat_1, feat_2 = k[1], k[3], k[5]
    embs_0_1 = embs[f"v_{feat_0}_1"]
    embs_0_2 = embs[f"v_{feat_0}_2"]
    embs_1_0 = embs[f"v_{feat_1}_0"]
    embs_1_2 = embs[f"v_{feat_1}_2"]
    embs_2_0 = embs[f"v_{feat_2}_0"]
    embs_2_1 = embs[f"v_{feat_2}_1"]
    bias_0 = embs[f"i_{feat_0}"][0]
    bias_1 = embs[f"i_{feat_1}"][0]
    bias_2 = embs[f"i_{feat_2}"][0]
    out = np.dot(embs_0_1, embs_1_0) + np.dot(embs_0_2, embs_2_0) + np.dot(embs_1_2, embs_2_1)
    out = out*(1/3)
    out += global_bias + (bias_0 + bias_1 + bias_2)*(1/np.sqrt(3))
    scores.append(sigmoid(out))

In [101]:
compare_scores = list(zip(scores, xlearn_scores))
print(compare_scores[:10])

[(0.268623177761137, 0.268623), (0.06099181027639442, 0.0609916), (0.08269272623407295, 0.0826926), (0.5479224014874199, 0.547922), (0.03862176018697644, 0.0386218), (0.2147861347995403, 0.214786), (0.12502738825164658, 0.125027), (0.49118174675686255, 0.491181), (0.03995730751212297, 0.0399572), (0.12661886084133878, 0.126619)]


### Scrap notes

In [62]:
u_p_embs = embs[f"v_3569360_1"]
u_c_embs = embs[f"v_3569360_2"]
p_u_embs = embs[f"v_60298_0"]
p_c_embs = embs[f"v_60298_2"]
c_u_embs = embs[f"v_3634989_0"]
c_p_embs = embs[f"v_3634989_1"]
u_bias = embs[f"i_3569360"]
p_bias = embs[f"i_60298"]
c_bias = embs[f"i_3634989"]

u_u_embs = embs[f"v_3569360_0"]
p_p_embs = embs[f"v_60298_1"]
c_c_embs = embs[f"v_3634989_2"]

In [63]:
out = np.dot(u_p_embs, p_u_embs)+np.dot(p_c_embs, c_p_embs)+np.dot(u_c_embs, c_u_embs)+u_bias+p_bias+c_bias+global_bias

In [91]:
embs_0_1

[0.218317,
 -0.40886,
 0.033446,
 0.0803355,
 0.0166485,
 0.249178,
 0.268246,
 0.347592,
 -0.471759,
 -0.10251,
 -0.891662,
 -0.0686677,
 0.0351942,
 0.139076,
 -0.243822,
 -0.622818,
 -0.0094951,
 0.0934188,
 -0.100039,
 -0.522374,
 -0.243693,
 -0.488498,
 -0.0956531,
 0.151277,
 -0.151594,
 -0.552921,
 -0.25861,
 -0.052828,
 0.103763,
 -0.375676,
 0.340832,
 -0.7]

In [65]:
out += np.dot(u_u_embs, u_u_embs)+np.dot(p_p_embs, p_p_embs)+np.dot(c_c_embs, c_c_embs)

In [66]:
tf.sigmoid(out)

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.99460341])>

In [40]:
embs[f"v_3569360_0"]

[0.0211019,
 0.091591,
 0.107846,
 0.0595417,
 0.0159502,
 0.0776743,
 0.0214465,
 0.0495851,
 0.100985,
 0.0164256,
 0.0172505,
 0.114423,
 0.114425,
 0.0282691,
 0.0274383,
 0.0659249,
 0.0761231,
 0.0855822,
 0.0407808,
 0.0677408,
 0.0279365,
 0.0379567,
 0.0888939,
 0.0468252,
 0.0345926,
 0.0184763,
 0.0651376,
 0.0287175,
 0.0964414,
 0.0745555,
 0.106452,
 0.0774059]

In [41]:
"v_60298_2"

'v_75561_2'

In [80]:
sigmoid(-5)

0.0066928509242848554

In [45]:
count = 0
for key, _ in embs.items():
    if("v_" in key and "_0" in key):
        count+=1

In [46]:
mapping.shape

(3637176, 2)

In [47]:
count

3637179

In [48]:
print("v_oadfioajf")

v_oadfioajf


In [58]:
save_line

'0 0:3569360:1 1:60298:1 2:3634989:1 \n'

In [81]:
lines[0].split(':')

['1 0', '3422065', '1 1', '95477', '1 2', '3633926', '1 ']

In [82]:
 k = lines[0].split(':')

In [83]:
k[1], k[3], k[5]

('3422065', '95477', '3633926')